In [3]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# Check folder files
%cd "/content/drive/Shareddrives/C23-PS139-ML"
%ls -al 

/content/drive/Shareddrives/C23-PS139-ML
total 442
drwx------ 2 root root   4096 May 27 14:59 csv/
drwx------ 2 root root   4096 May 30 12:48 data/
-rw------- 1 root root    275 May 29 13:30 env.env
-rw------- 1 root root   1370 May 31 05:43 install_chromium_driver.sh
drwx------ 2 root root   4096 May 25 09:07 .ipynb_checkpoints/
drwx------ 2 root root   4096 May 25 08:43 logs/
-rw------- 1 root root 122794 May 31 06:02 ner.ipynb
-rw------- 1 root root 148902 May 31 05:51 scraper.ipynb
-rw------- 1 root root 161913 May 25 06:18 selenium_worked_v1.1.ipynb


In [5]:
!apt-get -qq install -y dos2unix

# Remove any carriage return
!dos2unix -q "install_chromium_driver.sh"

# Run script
!bash "install_chromium_driver.sh"

Selecting previously unselected package dos2unix.
(Reading database ... 122545 files and directories currently installed.)
Preparing to unpack .../dos2unix_7.4.0-2_amd64.deb ...
Unpacking dos2unix (7.4.0-2) ...
Setting up dos2unix (7.4.0-2) ...
Processing triggers for man-db (2.9.1-1) ...
Executing: /tmp/apt-key-gpghome.UCfGPM8CpT/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.GkiI9g7uZs/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.tcJRu4Hb91/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gp

In [6]:
# Required libraries
!pip install -Uq selenium selectolax google-colab pandas supabase python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# !ps -ef | grep chrome

In [16]:
import ast
import json
import logging
import os
import random
import re
import secrets
import time
from datetime import datetime
from logging import Logger
from os import environ
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import pandas as pd
from dotenv import load_dotenv
from selectolax.parser import HTMLParser
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from supabase import Client, create_client
from tqdm.auto import tqdm


def get_webdriver() -> WebDriver:
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920, 1200")  # Fixed size
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    return driver


def get_logger(
    name: str,
    log_file: str = "log",
    log_folder: str = "logs",
    level: int = logging.INFO,
) -> Logger:
    """{log_folder}/{log_name}_{current_time}.log"""

    logger = logging.getLogger(name)
    # logger.setLevel(level)

    # Set logging level
    logging.getLogger("urllib3").setLevel(logging.WARNING)
    logging.getLogger("selenium").setLevel(logging.WARNING)

    formatter = logging.Formatter("%(asctime)s:%(levelname)s:%(name)s: %(message)s")

    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    # project_folder = Path(__file__).parent
    # logs_folder = project_folder / log_folder
    logs_folder = Path(log_folder)

    logs_folder.mkdir(parents=True, exist_ok=True)

    log_file = logs_folder / f"{log_file}_{current_time}.log"

    # print to file
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

    # log to stram
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.CRITICAL)
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    # Remove the default stream handler
    logger.propagate = False

    return logger


def wait_for_element(
    driver: WebDriver,
    by: By,
    value: str,
    timeout: int = 2,
    retry: int = 0,
    max_retry: int = 5,
    verbose: bool = False,
    logger=None,
) -> WebElement:
    try:
        element = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((by, value))
        )
        if verbose:
            logger.info(f"Found element: {value}")
        else:
            logger.debug(f"Found element: {value}")
        return element
    except TimeoutException as e:
        if retry >= max_retry:
            logger.error(f"Timeout for element: {value}")
            return None
        wait_time = 1**retry + secrets.randbelow(100) / 100
        if verbose:
            logger.debug(
                f"[{retry+1}/{max_retry}] Retrying element: {value}, retry in {wait_time} seconds."
            )
        else:
            logger.debug(
                f"[{retry+1}/{max_retry}] Retrying element: {value}, retry in {wait_time} seconds."
            )
        time.sleep(wait_time)
        return wait_for_element(
            driver, by, value, timeout, retry + 1, max_retry, verbose, logger
        )


def search_place(driver: WebDriver, keyword: str, city: str, logger=None) -> None:
    search_box = wait_for_element(driver, By.CSS_SELECTOR, "input#searchboxinput", logger=logger)
    search_keyword = " ".join(f"{keyword} {city}".split())
    search_box.send_keys(search_keyword + Keys.ENTER)
    logger.info(f"Searching for '{search_keyword}'")


def get_url(driver: WebDriver, url: str, logger=None):
    driver.get(url)
    logger.debug(f"Getting url: {url}")


def scrape_data(
    driver: WebDriver,
    keyword: str,
    city: str,
    url: str,
    max_found_places: int = 50,
    logger=None,
    keyword_category=None,
) -> pd.DataFrame:
    get_url(driver, url, logger=logger)
    search_place(driver, keyword, city, logger=logger)

    big_headline = wait_for_element(
        driver,
        By.CSS_SELECTOR,
        "h1.DUwDvf.fontHeadlineLarge",
        max_retry=2,
        logger=logger,
    )

    # Scroll to the bottom of the page
    logger.info("Scrolling google maps")

    search_keyword = " ".join(f"{keyword} {city}".split())

    with tqdm(total=max_found_places, desc=f"Scrolling into {search_keyword}") as tbar:
        actual_num_of_places = 0  # Track the actual number of places found

        while not big_headline:
            tree = HTMLParser(driver.page_source)

            a_tags = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")

            # Skip if no more places found
            if not a_tags:
                break

            # Scroll to the last element
            driver.execute_script("arguments[0].scrollIntoView();", a_tags[-1])

            try:
                # Look for last element
                WebDriverWait(driver, 10).until_not(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".lXJj5c.Hk4XGb"))
                )
            except TimeoutException as e:
                # Ignore
                pass

            end_of_list = tree.css_first(".m6QErb.tLjsW.eKbjU")

            if (max_found_places and (len(a_tags) >= max_found_places)) or end_of_list:
                logger.debug("Done scrolling.")
                tbar.total = len(a_tags)
                tbar.update(tbar.total - tbar.n)
                tbar.set_description(f"Done scrolling into {search_keyword}")
                break

            # Update the progress bar based on the actual number of places found
            tbar.total = max(len(a_tags), max_found_places)  # Update total value
            tbar.update(len(a_tags) - actual_num_of_places)
            actual_num_of_places = len(a_tags)

        # update to make the progress bar full
        tbar.total = len(a_tags)
        tbar.update(tbar.total - tbar.n)
    
    tree = HTMLParser(driver.page_source)
    if not big_headline:
        a_tags = tree.css("a.hfpxzc")
    else:
        a_tags = [tree.css_first("div.m6QErb.WNBkOb")]


    logger.info(f"Found {len(a_tags)} places.")

    # Scrape to all places found
    num_of_places = max(len(a_tags), max_found_places)

    pbar = tqdm(a_tags[:num_of_places], desc="Scraping", postfix="")
    for idx, a_tag in enumerate(a_tags[:num_of_places]):
        logger.debug(
            f"------------------- {idx + 1}/{num_of_places} -------------------"
        )

        df = pd.DataFrame()

        place_url = None
        if len(a_tags) > 1:
            place_url = a_tag.attributes["href"]
            get_url(driver, place_url, logger)
            wait_for_element(driver, By.CSS_SELECTOR, 'div[role="main"]', logger=logger)

        # ------------------- Overviews -------------------

        place_tree = HTMLParser(driver.page_source)
        name_element = wait_for_element(
            driver, By.CSS_SELECTOR, "h1.DUwDvf.fontHeadlineLarge", logger=logger
        )
        print(name_element, name_element.__dict__)
        # name = place_tree.css_first("h1").text(strip=True)
        try:
            name = name_element.text.strip()
        except:
            name = "no name"

        logger.debug(f"Name: {name}")
        pbar.set_description(f"Scraping {name}")

        # Wait until photos are loaded
        img_element = wait_for_element(
            driver,
            By.CSS_SELECTOR,
            "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd",
            timeout=3,
            max_retry=10,
            logger=logger,
        )

        img_url = None
        if img_element:
            place_tree = HTMLParser(driver.page_source)
            photos_element = place_tree.css_first(".RZ66Rb.FgCUCc button img")
            img_url = photos_element.attributes.get("src")

            logger.debug(f"Image URL: {img_url}")

        # ---------- Rating ----------
        rating_element = place_tree.css_first(".fontBodyMedium.dmRWX")

        if rating_element and rating_element.text(strip=True) != "":
            spans = rating_element.css("span")
            spans = [
                span.attributes.get("aria-label").strip()
                for span in spans
                if span.attributes.get("aria-label")
            ]
            if "bintang" in spans[0].lower():
                # Bintang 4,5
                rating = spans[0].removeprefix("bintang ").replace(",", ".")
                # 2.867 ulasan
                total_review = spans[0].removesuffix(" ulasan ").replace(".", ",")
            else:
                rating = float(spans[0].removesuffix(" stars"))

                if spans[1].endswith('reviews'):
                    total_review = int(spans[1].removesuffix(" reviews").replace(",", ""))
                elif spans[1].endswith('review'):
                    total_review = int(spans[1].removesuffix(" review").replace(",", ""))
        else:
            rating = None
            total_review = None

        # ---------- Price ----------
        if len(spans) > 2:
            # Harga: Sedang
            if "price: " in spans[2].lower():
                price = spans[2].removeprefix("Price: ")
            elif "harga: " in spans[2].lower():
                price = spans[2].removeprefix("Harga: ")
        else:
            price = None

        logger.debug(f"Rating: {rating}, Review Count: {total_review}, Price: {price}")

        # ---------- Category ----------
        category_element = place_tree.css_first("button.DkEaL")
        if category_element:
            category = category_element.text(strip=True)
        else:
            category = None
        logger.debug(f"Category: {category}")

        address_element = place_tree.css_first("div.Io6YTe.fontBodyMedium")
        if address_element:
            address = address_element.text(strip=True)
        else:
            address = ""
        logger.debug(f"Address: {address}")
        logger.debug(f"City: {city}")

        table = tree.css_first("table.eK4R0e.fontBodyMedium")
        opening_hours = {}
        if table:
            rows = table.css("tr")
            for row in rows:
                day = row.css_first("td").text(strip=True)
                hours = row.css_first(".mxowUb")
                hours = hours.attributes["aria-label"]
                hours = hours.replace("\u202f", "")
                opening_hours.update({day: hours})

            # JIR gak dimasukin

            logger.debug(f"Opening Hours: {opening_hours}")

        # ------------------- Reviews -------------------

        reviews_button = wait_for_element(
            driver, By.CSS_SELECTOR, "button.hh2c6:nth-child(2)", logger=logger
        )
        if reviews_button:
            reviews_button.click()

            detail_tree = HTMLParser(driver.page_source)
            review_categories_elements = detail_tree.css("button[role=radio].e2moi")
            review_categories = {}
            for review_categories_element in review_categories_elements[1:]:
                review_category_text_element = review_categories_element.css_first(
                    ".uEubGf.fontBodyMedium"
                )
                review_category_count_element = review_categories_element.css_first(
                    ".bC3Nkc.fontBodySmall"
                )
                review_category_text = (
                    review_category_text_element.text(strip=True)
                    .replace(" ", "_")
                    .lower()
                )
                review_category_count = review_category_count_element.text(strip=True)
                review_categories.update({review_category_text: review_category_count})

            logger.debug(f"Review Categories: {review_categories}")

        # ------------------- About -------------------
        about_button = wait_for_element(
            driver, By.CSS_SELECTOR, "button.hh2c6:nth-child(3)", logger=logger
        )
        about_dict = {}
        if about_button:
            about_button.click()

            clicked_button = driver.find_element(By.CSS_SELECTOR, "button.hh2c6.G7m0Af")
            data_tab_index = clicked_button.get_attribute("data-tab-index")
            if data_tab_index == "2":
                logger.debug("Clicked about button.")
            else:
                logger.error(
                    f"Failed to clicked about button, instead clicked {data_tab_index}"
                )

            wait_for_element(
                driver,
                By.CSS_SELECTOR,
                ".m6QErb.DxyBCb.kA9KIf.dS8AEf",
                timeout=10,
                max_retry=3,
                logger=logger,
            )

            # About button keknya
            wait_for_element(
                driver,
                By.CSS_SELECTOR,
                "div.iP2t7d",
                timeout=10,
                max_retry=3,
                logger=logger,
            )

            tree = HTMLParser(driver.page_source)
            divs = tree.css("div.iP2t7d")

            about_desc = tree.css_first("span.HlvSq")
            if about_desc:
                about_dict["description"] = about_desc.text(strip=True)

            for r_idx, div in enumerate(divs):
                about_category = (
                    div.css_first("h2.iL3Qke")
                    .text(strip=True)
                    .replace(" ", "_")
                    .lower()
                )
                li_elements = div.css("li")
                about_list = []
                about_no_list = []

                for li in li_elements:
                    img_elements = li.css("img")
                    src = img_elements[0].attributes.get("src")
                    about_element = li.css("span")
                    if src[-31:] == "not_interested_gm_grey_18dp.png":
                        about_no_list.append(about_element[0].text(strip=True))
                    else:
                        about_list.append(about_element[0].text(strip=True))
                if about_list:
                    about_dict[about_category] = about_list
                if about_no_list:
                    about_dict[f"No {about_category}"] = about_no_list

            logger.debug(f"About: {about_dict}")

        pbar.set_description(f"Scraped {name}")

        # ------------------- to dataframe -------------------

        if about_dict:
            about_dict = {
                key: ", ".join(val).lower().strip()
                if isinstance(val, list)
                else val.lower().strip()
                for key, val in about_dict.items()
            }
        

        place_df = pd.DataFrame(
            {
                "name": [name],
                "map_url": [place_url],
                "img_url": [img_url],
                "rating": [rating],
                "total_review": [total_review],
                "pricing": [price],
                "address": [address],
                "city": [city],
                "category": [json.dumps(review_categories)],
                "about": [json.dumps(about_dict)],
                "opening_hours": [opening_hours],
                "keyword_category": [[keyword_category]],
            }
        )

        df = pd.concat([df, place_df], ignore_index=True)

        # Untuk jaga2 mending save tiap iterasi dah
        # momen nungguin 15 menit tapi error di tengah jalan 💀
        pbar.set_description(f"Saving {name}")
        
        csv_dir = Path("/content") / ".tmp" / "csv"
        csv_dir.mkdir(parents=True, exist_ok=True)

        save_to_file(df, keyword=name, csv_folder=csv_dir)
        pbar.set_description(f"Saved {name}")

        pbar.update(1)



def save_to_file(df: pd.DataFrame, keyword: str, csv_folder: str = "csv", logger=None):
    csv_folder = Path(csv_folder)
    csv_folder.mkdir(parents=True, exist_ok=True)

    current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    keyword = keyword.strip().replace(" ", "_").lower()
    keyword = re.sub(r"\W+", "", keyword)  # Remove non-alphanumeric characters

    csv_filename = csv_folder / "{}_{}.csv".format(keyword, current_time)

    df.to_csv(csv_filename, index=False)


def upload_to_database(db_url: str, db_key: str, table_name: str, logger=None):
    supabase: Client = create_client(supabase_url=db_url, supabase_key=db_key)

    csv_dir = Path("/content") / ".tmp" / "csv"

    if not csv_dir.exists():
        logger.error(f"No folder {csv_dir} is found")
        return

    filenames = list(csv_dir.glob("*.csv"))

    if not filenames:
        logger.error(f"No csv files found in {csv_dir}")
        return

    # tbar = tqdm(filenames, desc="Reading csv files")
    with tqdm(filenames, desc="Reading csv files") as tbar:
        new_row_counter = 0
        update_row_counter = 0
        table = supabase.table(table_name)

        for filename in filenames:
            tbar.set_description(f"Reading {filename.name}")

            df_csv = pd.read_csv(filename)
            df_csv = df_csv.replace({np.NaN: None, np.nan: None})

            records = df_csv.to_dict("records")
            for row in records:
                try:
                    tbar.set_description(f"Inserting {filename.name} into database")
                    table.insert(row, upsert=True).execute()
                    new_row_counter += 1
                    tbar.set_description(f"Success inserted {filename.name} into database")
                    
                    os.remove(filename)                
                    tbar.set_description(f"Removed {filename.name}")
                except Exception as e:
                    error_dict = dict(ast.literal_eval(e.args[0]))

                    # duplicate handling
                    if error_dict.get("code") == "23505":
                        # fetch the existing row
                        matching_row = {
                            "name": row.get("name"),
                        }
                        existing_data, count = (
                            table.select("*").match(matching_row).execute()
                        )
                        if existing_data:
                            existing_row = existing_data[1][0]
                            existing_row.pop("id")

                            # update with old data (from db) 
                            # if new value (local) is None
                            if existing_row != row:
                                update_row_counter += 1
                                for key, value in row.items():
                                    existing_value = existing_row.get(key)

                                    if existing_value == value:
                                        continue

                                    if key == 'keyword_category' and existing_value is not None:
                                        existing_value = eval(existing_value)
                                        if isinstance(existing_value, set):
                                            existing_value = list(existing_value)

                                        value = eval(value) if value is not None else []
                                        if isinstance(value, set):
                                            value = list(value)
                                        
                                        value.extend(existing_value)
                                        row[key] = list(set(value))
                                    elif key == "keyword_category":
                                        value = eval(value) if value is not None else []
                                        if isinstance(value, set):
                                            value = list(value)
                                        row[key] = value
                                    else:
                                        row[key] = existing_value or value

                                # delete conflicting row
                                tbar.set_description("Deleting row in database")
                                table.delete().match(matching_row).execute()

                                # re-insert row
                                tbar.set_description("Re-insert with new row from local")
                                table.insert(row, upsert=True).execute()
                            else:
                                tbar.set_description(f"No changes")
                        
                        os.remove(filename) 
                        tbar.set_description(f"Removed {filename.name}")
                    else:
                        logger.error(e)

            tbar.update(1)
            
            tbar.set_postfix({"new rows": new_row_counter, "updated rows": update_row_counter})
    
    logger.info(f"Added {new_row_counter} rows")
    logger.info(f"Updated {update_row_counter} rows")


def run_scraper(
    driver: WebDriver,
    keyword: str,
    city: str,
    url: str = "https://www.google.com/maps/?hl=id",
    max_found_places: int = 50,
    csv_folder: str = "csv",
    table_name: str = "gmap_id",
    num: int = 1,
    logger=None,
    keyword_category=None,
):
    start_time = datetime.now()

    load_dotenv("env.env")

    scrape_data(driver, keyword, city, url, max_found_places, 
                logger=logger, keyword_category=keyword_category)
    upload_to_database(
        environ.get("DB_URL"), environ.get("DB_KEY"), table_name, logger=logger
    )

    driver.quit()

    elapsed_time = datetime.now() - start_time
    logger.info(f"Scraper-{num} elapsed: {elapsed_time}")


# Example usage:

keywords = [
    # ("Rekomendasi tempat wisata di", "tempat wisata"),
    # ("Rekomendasi tempat makan di", "tempat makan"),
    # ("Rekomendasi penginapan di", "penginapan"),
    ("Rekomendasi kegiatan di", "kegiatan"),
]

cities = [
    "Kabupaten Kepulauan Seribu",
    # "Kota Jakarta Barat",
    # "Kota Jakarta Pusat",
    # "Kota Jakarta Selatan",
    # "Kota Jakarta Timur",
    # "Kota Jakarta Utara",
    # "Kabupaten Bantul",
    # "Kabupaten Gunung Kidul",
    # "Kabupaten Kulon Progo",
    # "Kabupaten Sleman",
    # "Kota Yogyakarta",
]

keyword_city = [
    (keyword, keyword_category, city) 
    for city in  cities 
    for (keyword, keyword_category) in keywords
]


def scrape_keyword(keyword, city, max_found_places, num, keyword_category):
    logger = get_logger(f"scraper-{num}", log_folder="logs", level=logging.CRITICAL)
    driver = get_webdriver()
    run_scraper(
        driver=driver,
        keyword=keyword,
        city=city,
        max_found_places=max_found_places,
        num=num,
        logger=logger,
        keyword_category=keyword_category
    )




# with tqdm(keyword_city) as tbar:
#     for idx, (keyword, keyword_category, city) in enumerate(tbar):
#         tbar.set_description(f"{keyword} {city}")
#         scrape_keyword(keyword, city, 100, idx, keyword_category)

# multithreading
max_workers = os.cpu_count()

with tqdm(keyword_city, desc="Processing") as tbar:
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        
        for idx, (keyword, keyword_category, city) in enumerate(keyword_city):
            future = executor.submit(scrape_keyword, keyword, city, 100, idx, keyword_category)
            futures.append((future, keyword, city))

        # Process completed futures
        # for future, keyword, city in as_completed(futures):
        for completed_future in as_completed(futures):
            try:
                result = completed_future.result()
                # Process the result if needed
                future, keyword, city = next(f for f in futures if f[0] == completed_future)

                # Update the progress bar
                tbar.set_description(f"{keyword} {city}")
                tbar.update(1)
            except Exception as e:
                # Handle the exception
                print(f"An error occurred: {str(e)}")

Processing:   0%|          | 0/1 [00:00<?, ?it/s]

Scrolling into Rekomendasi kegiatan di Kabupaten Kepulauan Seribu:   0%|          | 0/100 [00:00<?, ?it/s]

Scraping:   0%|          | 0/29 [00:00<?, ?it/s]

Reading csv files:   0%|          | 0/29 [00:00<?, ?it/s]

AttributeError: ignored

'4.5'

# Archive

In [17]:
def upload_to_database(db_url: str, db_key: str, table_name: str, logger=None):
    supabase: Client = create_client(supabase_url=db_url, supabase_key=db_key)

    csv_dir = Path("/content") / ".tmp" / "csv"

    if not csv_dir.exists():
        logger.error(f"No folder {csv_dir} is found")
        return

    filenames = list(csv_dir.glob("*.csv"))

    if not filenames:
        logger.error(f"No csv files found in {csv_dir}")
        return

    # tbar = tqdm(filenames, desc="Reading csv files")
    with tqdm(filenames, desc="Reading csv files") as tbar:
        new_row_counter = 0
        update_row_counter = 0
        table = supabase.table(table_name)

        for filename in filenames:
            tbar.set_description(f"Reading {filename.name}")

            df_csv = pd.read_csv(filename)
            df_csv = df_csv.replace({np.NaN: None, np.nan: None})

            records = df_csv.to_dict("records")
            for row in records:
                try:
                    tbar.set_description(f"Inserting {filename.name} into database")
                    table.insert(row, upsert=True).execute()
                    new_row_counter += 1
                    tbar.set_description(f"Success inserted {filename.name} into database")
                    
                    os.remove(filename)                
                    tbar.set_description(f"Removed {filename.name}")
                except Exception as e:
                    error_dict = dict(ast.literal_eval(e.args[0]))

                    # duplicate handling
                    if error_dict.get("code") == "23505":
                        # fetch the existing row
                        matching_row = {
                            "name": row.get("name"),
                        }
                        existing_data, count = (
                            table.select("*").match(matching_row).execute()
                        )
                        if existing_data:
                            existing_row = existing_data[1][0]
                            existing_row.pop("id")

                            # update with old data (from db) 
                            # if new value (local) is None
                            if existing_row != row:
                                update_row_counter += 1
                                for key, value in row.items():
                                    existing_value = existing_row.get(key)

                                    if existing_value == value:
                                        continue

                                    if key == 'keyword_category' and existing_value is not None:
                                        existing_value = eval(existing_value)
                                        if isinstance(existing_value, set):
                                            existing_value = list(existing_value)

                                        value = eval(value) if value is not None else []
                                        if isinstance(value, set):
                                            value = list(value)
                                        
                                        value.extend(existing_value)
                                        row[key] = list(set(value))
                                    elif key == "keyword_category":
                                        value = eval(value) if value is not None else []
                                        if isinstance(value, set):
                                            value = list(value)
                                        row[key] = value
                                    else:
                                        row[key] = existing_value or value

                                # delete conflicting row
                                tbar.set_description("Deleting row in database")
                                table.delete().match(matching_row).execute()

                                # re-insert row
                                tbar.set_description("Re-insert with new row from local")
                                table.insert(row, upsert=True).execute()
                            else:
                                tbar.set_description(f"No changes")
                        
                        os.remove(filename) 
                        tbar.set_description(f"Removed {filename.name}")
                    else:
                        logger.error(e)

            tbar.update(1)
            
            tbar.set_postfix({"new rows": new_row_counter, "updated rows": update_row_counter})
    
    logger.info(f"Added {new_row_counter} rows")
    logger.info(f"Updated {update_row_counter} rows")

upload_to_database(
    environ.get("DB_URL"), environ.get("DB_KEY"), "gmap_id", logger=get_logger("aw")
)

Reading csv files:   0%|          | 0/27 [00:00<?, ?it/s]

In [ ]:
# from tqdm.auto import tqdm
# import time

# items = list(range(1, 1001))
# with tqdm(total=len(items), desc="Processing") as tbar:
#     for item in items:
#         time.sleep(0.01)
#         tbar.set_description(f'Processing {item}th item')
#     tbar.set_description(f'Done. Found {tbar.total} items')


In [ ]:
# enaknya kode klo uda functional, tinggal dipanggil bg
# load_dotenv('env.env')
# upload_to_database(
#     environ.get("DB_URL"), environ.get("DB_KEY"), "gmap_id", logger=get_logger('aw')
# )

In [ ]:
# import ast
# import json
# import random
# import secrets
# import time
# from datetime import datetime
# from os import environ

# import pandas as pd
# from dotenv import load_dotenv
# from selectolax.parser import HTMLParser
# from selenium import webdriver
# from selenium.common.exceptions import NoSuchElementException, TimeoutException
# from selenium.webdriver.chrome.webdriver import WebDriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.remote.webelement import WebElement
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.wait import WebDriverWait
# from supabase import Client, create_client
# from tqdm.notebook import tqdm
# import logging
# from pathlib import Path

# start_time = datetime.now()


# def get_webdriver():
#     options = webdriver.ChromeOptions()
#     options.add_argument("--verbose")
#     options.add_argument("--no-sandbox")
#     options.add_argument("--headless")
#     options.add_argument("--disable-gpu")
#     options.add_argument("--window-size=1920, 1200")  # Fixed size
#     options.add_argument("--disable-dev-shm-usage")
#     driver = webdriver.Chrome(options=options)
#     return driver


# # Setup logging
# def get_logger(name: str, log_file: str = "log", log_folder: str = "logs"):
#     """{log_folder}/{log_name}_{current_time}.log"""

#     logger = logging.getLogger(name)
#     logger.setLevel(logging.INFO)

#     formatter = logging.Formatter("%(asctime)s:%(levelname)s:%(name)s: %(message)s")

#     current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

#     # project_folder = Path(__file__).parent

#     # logs_folder = project_folder / log_folder
#     logs_folder = Path(log_folder)

#     logs_folder.mkdir(parents=True, exist_ok=True)

#     log_file = logs_folder / f"{log_file}_{current_time}.log"

#     # print to file
#     file_handler = logging.FileHandler(log_file)
#     file_handler.setFormatter(formatter)

#     logger.addHandler(file_handler)

#     # Remove the default stream handler
#     logger.propagate = True

#     return logger


# # Initialize driver
# driver = get_webdriver()

# # Initialize logger
# logger = get_logger("scrapper")

# # Initialize dataframe
# df = pd.DataFrame()

# # Set logging level
# logging.getLogger("urllib3").setLevel(logging.WARNING)
# logging.getLogger("selenium").setLevel(logging.WARNING)

# logger.setLevel(logging.ERROR)  # Modify this


# def get_url(driver: WebDriver, url: str):
#     driver.get(url)
#     logger.info(f"Getting url: {url}")


# # Configurations

# url = "https://www.google.com/maps/?hl=en"
# wait_timeout = 10

# max_found_places = 50
# num_of_places = max_found_places

# # max_found_review = 50
# city = "jakarta barat"
# keyword = f"Rekomendasi aktivitas di {city}"


# driver.get(url)


# def wait_for_element(
#     by: By,
#     value: str,
#     timeout: int = 2,
#     retry: int = 0,
#     max_retry: int = 5,
#     verbose: bool = False,
# ) -> WebElement:
#     try:
#         element = WebDriverWait(driver, timeout).until(
#             EC.presence_of_element_located((by, value))
#         )
#         if verbose:
#             logger.info(f"Found element: {value}")
#         else:
#             logger.debug(f"Found element: {value}")
#         return element
#     except TimeoutException:
#         if retry >= max_retry:
#             logger.error(f"Timeout for element: {value}")
#             return None
#         # wait_time = 2**retry + random.uniform(0, 1)
#         # or this one
#         # capek nunggunya, set ke 1 aja
#         wait_time = 1**retry + secrets.randbelow(100) / 100
#         if verbose:
#             logger.debug(
#                 f"[{retry+1}/{max_retry}] Retrying element: {value}, retry in {wait_time} seconds."
#             )
#         else:
#             logger.debug(
#                 f"[{retry+1}/{max_retry}] Retrying element: {value}, retry in {wait_time} seconds."
#             )
#         time.sleep(wait_time)
#         wait_for_element(by, value, timeout, retry + 1, max_retry)


# def search_place(keyword: str):
#     search_box = wait_for_element(By.CSS_SELECTOR, "input#searchboxinput")
#     search_box.send_keys(keyword + Keys.ENTER)
#     logger.info(f"Searching for '{keyword}'")


# search_place(keyword)


# big_headline = wait_for_element(
#     By.CSS_SELECTOR, "h1.DUwDvf.fontHeadlineLarge", max_retry=2
# )

# # Scroll to the bottom of the page
# while not big_headline:
#     tree = HTMLParser(driver.page_source)

#     a_tags = driver.find_elements(By.CSS_SELECTOR, "a.hfpxzc")

#     # Skip if no more places found
#     if not a_tags:
#         break

#     # Scroll to the last element
#     driver.execute_script("arguments[0].scrollIntoView();", a_tags[-1])

#     try:
#         # Look for last element
#         WebDriverWait(driver, 10).until_not(
#             EC.presence_of_element_located((By.CSS_SELECTOR, ".lXJj5c.Hk4XGb"))
#         )
#     except TimeoutException as e:
#         # Ignore
#         pass

#     end_of_list = tree.css_first(".m6QErb.tLjsW.eKbjU")

#     if (max_found_places and len(a_tags) >= max_found_places) or end_of_list:
#         logger.debug("Done scrolling.")
#         break


# tree = HTMLParser(driver.page_source)

# if not big_headline:
#     a_tags = tree.css("a.hfpxzc")
# else:
#     a_tags = [tree.css_first("div.m6QErb.WNBkOb")]

# logger.info(f"Found {len(a_tags)} places.")

# # Scrape to all places found
# num_of_places = min(len(a_tags), max_found_places)
# for idx, a_tag in enumerate(tqdm(a_tags[:num_of_places], desc="Scraping gmap")):
#     logger.info(f"------------------- {idx + 1}/{num_of_places} -------------------")

#     if len(a_tags) > 1:
#         place_url = a_tag.attributes["href"]
#         get_url(driver, place_url)
#         wait_for_element(By.CSS_SELECTOR, 'div[role="main"]')

#     # ------------------- Overviews -------------------

#     place_tree = HTMLParser(driver.page_source)
#     name = place_tree.css_first("h1").text(strip=True)
#     logger.info(f"Name: {name}")

#     # Wait until photos are loaded
#     img_element = wait_for_element(
#         By.CSS_SELECTOR, "button.aoRNLd.kn2E5e.NMjTrf.lvtCsd", timeout=3, max_retry=10
#     )

#     img_url = None
#     if img_element:
#         place_tree = HTMLParser(driver.page_source)
#         photos_element = place_tree.css_first(".RZ66Rb.FgCUCc button img")
#         img_url = photos_element.attributes.get("src")

#         logger.info(f"Image URL: {img_url}")

#     # # Open the share dialog
#     # driver.find_element(By.CSS_SELECTOR, "button[data-value='Share']").click()

#     # # Wait for the url input to appear
#     # map_url_element = wait_for_element(By.CSS_SELECTOR, "input.vrsrZe[value]", timeout=5, max_retry=3)

#     # if not map_url_element:
#     #     map_url = ""

#     # while map_url_element:
#     #     map_tree = HTMLParser(driver.page_source)
#     #     map_url = map_tree.css_first("input.vrsrZe").attributes.get("value")
#     #     if map_url:
#     #         break

#     # logger.info(f"Map URL: {map_url}")

#     # # Close the share dialog
#     # driver.find_element(By.CSS_SELECTOR, "button[aria-label='Close']").click()

#     # ---------- Rating ----------
#     rating_element = place_tree.css_first(".fontBodyMedium.dmRWX")

#     if rating_element and rating_element.text(strip=True) != "":
#         spans = rating_element.css("span")
#         spans = [
#             span.attributes.get("aria-label").strip()
#             for span in spans
#             if span.attributes.get("aria-label")
#         ]
#         rating = float(spans[0].removesuffix(" stars"))
#         total_review = int(spans[1].removesuffix(" reviews").replace(",", ""))
#     else:
#         rating = None
#         total_review = None

#     # ---------- Price ----------
#     if len(spans) > 2:
#         price = spans[2].removeprefix("Price: ")
#     else:
#         price = None

#     logger.info(f"Rating: {rating}, Review Count: {total_review}, Price: {price}")

#     # ---------- Category ----------
#     category_element = place_tree.css_first("button.DkEaL")
#     if category_element:
#         category = category_element.text(strip=True)
#     else:
#         category = None
#     logger.info(f"Category: {category}")

#     address_element = place_tree.css_first("div.Io6YTe.fontBodyMedium")
#     if address_element:
#         address = address_element.text(strip=True)
#     else:
#         address = ""
#     logger.info(f"Address: {address}")
#     logger.info(f"City: {city}")

#     table = tree.css_first("table.eK4R0e.fontBodyMedium")
#     if table:
#         rows = table.css("tr")
#         opening_hours = {}
#         for row in rows:
#             day = row.css_first("td").text(strip=True)
#             hours = row.css_first(".mxowUb")
#             hours = hours.attributes["aria-label"]
#             hours = hours.replace("\u202f", "")
#             opening_hours.update({day: hours})

#         print(f"Opening Hours: {opening_hours}")

#     # ------------------- Reviews ------------------- keknya gak jadi dipake

#     reviews_button = driver.find_element(By.CSS_SELECTOR, "button.hh2c6:nth-child(2)")
#     if reviews_button:
#         reviews_button.click()

#         detail_tree = HTMLParser(driver.page_source)
#         review_categories_elements = detail_tree.css("button[role=radio].e2moi")
#         review_categories = {}
#         for review_categories_element in review_categories_elements[1:]:
#             review_category_text_element = review_categories_element.css_first(
#                 ".uEubGf.fontBodyMedium"
#             )
#             review_category_count_element = review_categories_element.css_first(
#                 ".bC3Nkc.fontBodySmall"
#             )
#             review_category_text = (
#                 review_category_text_element.text(strip=True).replace(" ", "_").lower()
#             )
#             review_category_count = review_category_count_element.text(strip=True)
#             review_categories.update({review_category_text: review_category_count})

#         logger.info(f"Review Categories: {review_categories}")

#         # while True:
#         #     wait_for_element(By.CSS_SELECTOR, "div.jJc9Ad", timeout=10, max_retry=1)

#         #     divs = driver.find_elements(By.CSS_SELECTOR, "div.jJc9Ad")
#         #     driver.execute_script("arguments[0].scrollIntoView();", divs[-1])

#         #     if len(divs) >= min(total_review, max_found_review):
#         #         logger.debug("Done scrolling.")
#         #         break

#         # tree = HTMLParser(driver.page_source)
#         # divs = tree.css("div.jJc9Ad")

#         # logger.info(f"Found {len(divs)} reviews")

#         # for r_idx, div in enumerate(divs):
#         #     reviewer_name = div.css_first("div.d4r55").text(strip=True)
#         #     reviewer_badge_total_review = div.css_first("div.RfnDt")
#         #     if not reviewer_badge_total_review:
#         #         reviewer_badge_total_review = ""
#         #     else:
#         #         reviewer_badge_total_review = reviewer_badge_total_review.text(strip=True)

#         #         # Local Guide · 218 reviews
#         #         if "·" in reviewer_badge_total_review:
#         #             (
#         #                 reviewer_badge,
#         #                 reviewer_total_review,
#         #             ) = reviewer_badge_total_review.split("·")
#         #         else:
#         #             reviewer_badge = ""
#         #             reviewer_total_review = reviewer_badge_total_review

#         #     reviewer_badge = reviewer_badge.strip()
#         #     reviewer_total_review = reviewer_total_review.replace("reviews", "").strip()

#         #     reviewer_star = div.css_first("span.kvMYJc").attributes.get("aria-label")

#         #     # remove star or stars
#         #     if reviewer_star.endswith("stars"):
#         #         reviewer_star = reviewer_star.removesuffix("stars").strip()
#         #     else:
#         #         reviewer_star = reviewer_star.removesuffix("star").strip()

#         #     num_of_stars = float(reviewer_star)
#         #     when_review = div.css_first("span.rsqaWe").text(strip=True)

#         #     review_description = div.css_first("div.MyEned")
#         #     if not review_description:
#         #         review_description = ""
#         #     else:
#         #         review_description_id = review_description.id

#         #         while True:
#         #             review_element = driver.find_element(By.ID, review_description_id)
#         #             driver.execute_script("arguments[0].scrollIntoView();", review_element)
#         #             try:
#         #                 more_button = review_element.find_element(
#         #                     By.CSS_SELECTOR, "button.w8nwRe.kyuRq"
#         #                 )
#         #                 more_button.click()
#         #             except NoSuchElementException as e:
#         #                 break

#         #         review_description_tree = HTMLParser(driver.page_source)
#         #         review_description = review_description_tree.css_first(
#         #             f"#{review_description_id}"
#         #         ).text(strip=True)

#         #     reviewer_dict = {
#         #         "name": reviewer_name,
#         #         "badge": reviewer_badge,
#         #         "total_review": reviewer_total_review,
#         #         "num_of_stars": num_of_stars,
#         #         "when_review": when_review,
#         #         "review_description": review_description,
#         #     }

#         #     logger.debug(f"{r_idx+1}/{len(divs)} Reviewer: {reviewer_dict}")

#     # ------------------- About -------------------
#     about_button = wait_for_element(By.CSS_SELECTOR, "button.hh2c6:nth-child(3)")
#     if about_button:
#         about_button.click()

#         clicked_button = driver.find_element(By.CSS_SELECTOR, "button.hh2c6.G7m0Af")
#         data_tab_index = clicked_button.get_attribute("data-tab-index")
#         if data_tab_index == "2":
#             logger.debug("Clicked about button.")
#         else:
#             logger.error(
#                 f"Failed to clicked about button, instead clicked {data_tab_index}"
#             )

#         wait_for_element(
#             By.CSS_SELECTOR, ".m6QErb.DxyBCb.kA9KIf.dS8AEf", timeout=10, max_retry=3
#         )

#         # About button keknya
#         wait_for_element(By.CSS_SELECTOR, "div.iP2t7d", timeout=10, max_retry=3)

#         tree = HTMLParser(driver.page_source)
#         divs = tree.css("div.iP2t7d")

#         about_dict = {}

#         about_desc = tree.css_first("span.HlvSq")
#         if about_desc:
#             about_dict["description"] = about_desc.text(strip=True)

#         for r_idx, div in enumerate(divs):
#             about_category = (
#                 div.css_first("h2.iL3Qke").text(strip=True).replace(" ", "_").lower()
#             )
#             li_elements = div.css("li")
#             about_list = []
#             about_no_list = []

#             for li in li_elements:
#                 img_elements = li.css("img")
#                 src = img_elements[0].attributes.get("src")
#                 about_element = li.css("span")
#                 if src[-31:] == "not_interested_gm_grey_18dp.png":
#                     about_no_list.append(about_element[0].text(strip=True))
#                 else:
#                     about_list.append(about_element[0].text(strip=True))
#             if about_list:
#                 about_dict[about_category] = about_list
#             if about_no_list:
#                 about_dict[f"No {about_category}"] = about_no_list

#         logger.info(f"About: {about_dict}")

#     # ------------------- to dataframe -------------------

#     # Alt solution: as dict

#     about_dict = {
#         key: ", ".join(val).lower().strip()
#         if isinstance(val, list)
#         else val.lower().strip()
#         for key, val in about_dict.items()
#     }

#     place_df = pd.DataFrame(
#         {
#             "name": [name],
#             "map_url": [place_url],
#             "img_url": [img_url],
#             "rating": [rating],
#             "total_review": [total_review],
#             "pricing": [price],
#             "address": [address],
#             "city": [city],
#             "category": [json.dumps(review_categories)],
#             "about": [json.dumps(about_dict)],
#         }
#     )

#     # # Solution 1: deprecated
#     # for key,val in review_categories.items():
#     #     cat_key = f'category_{key.strip().lower().replace(" ", "_")}'
#     #     place_df[cat_key] = val

#     # for key, val in about_dict.items():
#     #     about_key = f'about_{key.strip().lower().replace(" ", "_")}'
#     #     about_val = ','.join(val).strip().lower() if type(val) == list else val
#     #     place_df[about_key] = about_val

#     # # Solution 2
#     # categories = '; '.join(
#     #     f"{key.replace(' ', '_').lower()}:{val.lower()}"
#     #     for key,val in review_categories.items()
#     # )

#     # abouts = '; '.join(
#     #     "{}:{}".format(
#     #         key.replace(' ', '_').lower(),
#     #         ','.join(val).strip().lower() if type(val) == list else val
#     #     )
#     #     for key,val in about_dict.items()
#     # )

#     # place_df['categories'] = categories
#     # place_df['abouts'] = abouts

#     df = pd.concat([df, place_df], ignore_index=True)

#     # break

# # ------------------- saving to file -------------------

# csv_folder = Path("csv")
# csv_folder.mkdir(parents=True, exist_ok=True)

# current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# csv_filename = csv_folder / "{}_{}.csv".format(
#     keyword.replace(" ", "_").lower(), current_time
# )
# df.to_csv(csv_filename, index=False)


# # ------------------- upload to database ---------------------
# load_dotenv("env.env")

# supabase: Client = create_client(
#     supabase_url=environ.get("DB_URL"), supabase_key=environ.get("DB_KEY")
# )

# records = df.to_dict("records")
# table = supabase.table("gmap_id")
# new_row_counter = 0
# update_row_counter = 0
# for row in tqdm(records, desc="Updating database"):
#     try:
#         table.insert(row, upsert=True).execute()
#         new_row_counter += 1
#     except Exception as e:
#         error_dict = dict(ast.literal_eval(e.args[0]))

#         # duplicate handling
#         if error_dict.get("code") == "23505":
#             details = error_dict["details"].split("Key (")[1].split(")=(")
#             keys = [key.strip() for key in details[0].split(",")]
#             values = [value.strip() for value in details[1].split(")")[0].split(",")]
#             detail_dict = {key: value for key, value in zip(keys, values)}

#             # fetch the existing row
#             existing_data, count = table.select("*").match(detail_dict).execute()
#             existing_row = existing_data[1][0]
#             existing_row.pop("id")

#             # update with old data if new value is None
#             if existing_row != row:
#                 update_row_counter += 1
#                 for key, value in row.items():
#                     if value is None:
#                         row[key] = existing_row.get(key)

#             # delete conflicting row
#             table.delete().match(detail_dict).execute()

#             # re-insert row
#             table.insert(row, upsert=True).execute()

# print(f"Added {new_row_counter} rows")
# print(f"Updated {update_row_counter} rows")

# driver.quit()

# print(f"Time ellapsed: {datetime.now() - start_time}")

In [ ]:
df.head()

In [ ]:
df.columns